In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import efficientnet_b0
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import KFold, GroupKFold, train_test_split
from datetime import datetime
from tqdm import tqdm
import albumentations as A
import gc
import matplotlib.pyplot as plt
import math
import multiprocessing
import os
import time
import random
from typing import List, Tuple, Union
import pickle
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Kaggle
# DATA_ROOT = "/kaggle/input/hms-harmful-brain-activity-classification/"
# TRAIN_EEG = "train_eegs"
# TRAIN_SPEC = "train_spectrograms"
# TEST_EEG = "test_eegs"
# TEST_SPEC = "test_spectrograms"

#Local
DATA_ROOT = "/home/benluo/HBAC/data/hbac"
TRAIN_EEG = "train_eegs"
TRAIN_SPEC = "train_spectrograms"
TEST_EEG = "test_eegs"
TEST_SPEC = "test_spectrograms"

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
train_list = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"))

train_list_reduced = train_list.iloc[:train_list.shape[0]//2,:]

train_list_reduced.to_csv(os.path.join(DATA_ROOT, "train_reduced.csv"))

In [5]:
def generate_data(val_folds:int = 5,
                  test_size:float = 0.2,
                  saved_spec:str = "all_spec.pkl",
                  saved_eeg:str = "all_eeg.pkl"):

    train_list = pd.read_csv(os.path.join(DATA_ROOT, "train_reduced.csv"))

    print("All data")
    display(train_list.head())

    # test_df = pd.read_csv(os.path.join(DATA_ROOT, "test.csv"))

    label_cols = train_list.columns[-6:]
    

    #Select first eeg and spectrogram in each spectrogram file
    train_df = train_list[['spectrogram_id','eeg_id','patient_id','spectrogram_label_offset_seconds','eeg_label_offset_seconds']]

    aux = train_list[label_cols].copy()
    
    for label in label_cols:
        train_df[label] = aux[label].values
        
    y_data = train_df[label_cols].values
    y_data = y_data / y_data.sum(axis=1,keepdims=True)
    train_df[label_cols] = y_data

    aux = train_list['expert_consensus'].copy()
    train_df['target'] = aux
    train_df = train_df.reset_index()

    train_df = train_df.sort_values("patient_id")

    test_df = train_df.iloc[int(0.9*train_df.shape[0]):]

    train_df = train_df.iloc[:int(0.9*train_df.shape[0])]

    train_df = train_df.reset_index()
    test_df = test_df.reset_index()

    print("Training data")
    display(train_df.head())

    print("Testing data")
    display(test_df.head())


    gkf = GroupKFold(n_splits=val_folds)

    #KFold grouped by patient ID
    for fold, (train_index, val_index) in enumerate(gkf.split(train_df, train_df.target, train_df.patient_id)):
        train_df.loc[val_index, "fold"] = int(fold)
    
    all_eeg = {}
    
    all_spec = {}


    try:

        with open(os.path.join(DATA_ROOT,saved_spec), "rb") as handle:
            all_spec = pickle.load(handle)

        with open(os.path.join(DATA_ROOT,saved_eeg), "rb") as handle:
            all_eeg = pickle.load(handle)

    except:
    
        for idx, row in tqdm(train_list.iterrows()):

            spec_id = row["spectrogram_id"]
            eeg_id = row["eeg_id"]

            if spec_id not in all_spec:

                spec = pd.read_parquet(os.path.join(DATA_ROOT, TRAIN_SPEC, str(spec_id)+".parquet"))

                all_spec[spec_id] = spec.iloc[:,1:].values.astype(dtype=np.float32)

            if eeg_id not in all_eeg:

                eeg = pd.read_parquet(os.path.join(DATA_ROOT, TRAIN_EEG, str(eeg_id)+".parquet"))

                all_eeg[eeg_id] = eeg.iloc[:,1:].values.astype(dtype=np.float32)
        
        with open(os.path.join(DATA_ROOT, saved_eeg), "wb") as handle:
            pickle.dump(all_eeg, handle)
        
        with open(os.path.join(DATA_ROOT, saved_spec), "wb") as handle:
            pickle.dump(all_spec, handle)
    
    # for i, row in tqdm(test_df.iterrows()):
            
    #     eeg_id = row["eeg_id"]
    #     spec_id = row["spectrogram_id"]

    #     if eeg_id not in test_eeg:

    #         eeg = pd.read_parquet(os.path.join(DATA_ROOT, TRAIN_EEG, str(eeg_id)+".parquet"))


    #     if spec_id not in test_spec:
    
    #         spec = pd.read_parquet(os.path.join(DATA_ROOT, TRAIN_SPEC, str(spec_id)+".parquet"))
        
    #     test_eeg[eeg_id] = eeg.iloc[:,1:].values.astype(dtype=np.float16)
    #     test_spec[spec_id] = spec.iloc[:,1:].values.astype(dtype=np.float16)
    
    return train_df, test_df, all_eeg, all_spec, label_cols

In [6]:
train_df, test_df, all_eeg, all_spec, label_cols = generate_data()

All data


,Unnamed: 0,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


Training data


/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,level_0,index,spectrogram_id,eeg_id,patient_id,spectrogram_label_offset_seconds,eeg_label_offset_seconds,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,41744,41744,802850878,1873660287,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
1,49045,49045,957002006,165634434,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
2,41746,41746,802850878,2057577408,56,290.0,46.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
3,41745,41745,802850878,2057577408,56,244.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
4,25278,25278,497667405,374550767,56,694.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other


Testing data


,level_0,index,spectrogram_id,eeg_id,patient_id,spectrogram_label_offset_seconds,eeg_label_offset_seconds,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,5410,5410,91996359,3686473557,57272,122.0,122.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
1,5389,5389,91996359,3686473557,57272,72.0,72.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
2,5384,5384,91996359,3686473557,57272,60.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
3,5394,5394,91996359,3686473557,57272,82.0,82.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
4,5385,5385,91996359,3686473557,57272,62.0,62.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA


In [7]:
#data processing
class HMSDataset(Dataset):

    def __init__(self,
                 df:pd.DataFrame = None,
                 aug: bool = False) -> None:
        

        super(HMSDataset, self).__init__()

        self.df = df

        #Data augmentation
        self.aug = aug

        self.eeg_sample_freq = 200 # 200 Hz
        self.spec_sample_freq = 0.5 # 0.5 Hz
    
        
        #data augmentation
        self.transforms = A.Compose([
            A.HorizontalFlip(p=0.5)
        ])

    def format_data(self, eeg:np.array, spec:np.array) -> Tuple[torch.tensor]:

        eps = 1e-6

        eeg = eeg.astype(dtype=np.float32)
        spec = spec.astype(dtype=np.float32)

        #Normalizing and getting rid of Nans
        eeg_mean = np.nanmean(eeg.flatten())
        eeg_std = np.nanstd(eeg.flatten())
        eeg = (eeg-eeg_mean)/(eeg_std+eps)
        eeg = np.nan_to_num(eeg, nan=0.0)

        #Normalizing and getting rid of Nans
        spec = np.clip(spec, np.exp(-4), np.exp(8))
        spec = np.log(spec)
        
        spec_mean = np.nanmean(spec.flatten())
        spec_std = np.nanstd(spec.flatten())
        spec = (spec-spec_mean)/(spec_std+eps)
        spec = np.nan_to_num(spec, nan=0.0)
        
        if self.aug:

            eeg = self.transforms(image=eeg)["image"]
            spec = self.transforms(image=spec)["image"]

        eeg = torch.tensor(eeg.copy())
        spec = torch.tensor(spec.copy())

        return eeg, spec


    def __getitem__(self, index) -> dict:
    
        row = self.df.iloc[index]

        eeg_id = row["eeg_id"]
        spec_id = row["spectrogram_id"]

        #EEG Sub-sampling
        start = int(row["eeg_label_offset_seconds"]*self.eeg_sample_freq)
        end = int((row["eeg_label_offset_seconds"]+50)*self.eeg_sample_freq)
        eeg = all_eeg[eeg_id][start:end]
        
        #Spectrogram Sub-sampling
        start = int(row["spectrogram_label_offset_seconds"]*self.spec_sample_freq)
        end = int((row["spectrogram_label_offset_seconds"]+600)*self.spec_sample_freq)
        spec = all_spec[spec_id][start:end].T

        # print(all_eeg[eeg_id].shape, eeg.shape, all_spec[spec_id].shape, spec.shape)

        eeg, spec = self.format_data(eeg, spec)

        label = torch.tensor(row[label_cols], dtype=torch.float32)

        return eeg, spec, label


    def __len__(self):

        return self.df.shape[0]

    @staticmethod
    def collate_fn(batch):

        eeg, spec, label = zip(*batch)
        
        if eeg is not None:
            eeg = torch.stack(eeg, dim=0).float().unsqueeze(1)
        
        if spec is not None:
            spec = torch.stack(spec, dim=0).float().unsqueeze(1).expand(-1,3,-1,-1)

        if label is not None:            
            label = torch.stack(label, dim=0)
        
        return {
            "eeg": eeg,
            "spec": spec,
            "label": label
        }
    

In [8]:
class CNNDetector(nn.Module):

    def __init__(self, config={}):

        super(CNNDetector, self).__init__()

        self.backbone_1d = None

        self.backbone_2d = efficientnet_b0(weights="IMAGENET1K_V1")
        
#         self.backbone_2d.conv1 = nn.Conv2d(config.get("in_channels",1), 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
        
        self.backbone_2d.classifier[1] = nn.Linear(1280, config.get("num_classes", 6))
        
#         print(self.backbone_2d)

        self.head = None

    def get_loss(self, pred_dict):

        pass

    def forward(self, data_dict):

        """
        returns a dictionary pred_dict with the logits for loss calculation and gradient descent.
        """
        
#         print(torch.isnan(torch.sum(data_dict["spec"])))

        return self.backbone_2d(data_dict["spec"])

In [9]:
dataset = HMSDataset(df=train_df)
batch_size = 32

print([data.shape for data in dataset[0]])

i = np.random.randint(len(dataset))
print(dataset[i])

[torch.Size([10000, 19]), torch.Size([400, 300]), torch.Size([6])]
(tensor([[-1.2356, -0.5683, -0.4377,  ..., -0.5966, -1.3976, -0.5510],
        [-0.8868, -0.1387, -0.0332,  ..., -0.5353, -1.2706, -0.2433],
        [-0.6432,  0.2154,  0.3555,  ..., -0.3242, -0.8933,  0.2023],
        ...,
        [ 0.8540,  0.7787,  0.3340,  ...,  0.2211,  0.1655,  0.9661],
        [ 2.2337,  1.9746,  1.5141,  ...,  1.4108,  1.4537, -0.2520],
        [-0.4695, -1.0579, -1.6340,  ..., -0.5016, -1.2094, -1.1652]]), tensor([[ 1.6868,  1.7155,  1.9145,  ...,  2.0851,  1.7474,  1.8000],
        [ 1.8237,  1.8382,  1.9104,  ...,  2.1552,  1.9710,  1.9838],
        [ 1.8580,  1.9329,  2.0158,  ...,  2.2104,  2.0888,  1.9773],
        ...,
        [-1.3422, -1.3793, -1.2765,  ..., -1.4646, -0.9663, -0.9812],
        [-1.2196, -1.4646, -1.3422,  ..., -1.5717, -0.8624, -0.8741],
        [-1.3081, -1.4646, -1.4646,  ..., -1.5148, -0.9112, -0.8402]]), tensor([1., 0., 0., 0., 0., 0.]))


In [10]:
#testing

@torch.no_grad()
def validate(model, valid_dataloader):

    model.eval()

    loss_fn = nn.KLDivLoss(reduction="batchmean")

    loss = torch.tensor([0.]).to(device)

    for batch in tqdm(valid_dataloader):

        for key in batch:

            batch[key] = batch[key].to(device)

        predictions = model(batch)
        
        predictions = F.log_softmax(predictions,dim=1)

        loss += loss_fn(predictions,batch["label"])

    loss = loss / len(valid_dataloader)

    model.train()

    print("Validation loss", loss.item())

    return loss.item()

def test(model, test_dataloader):

    model.eval()

    correct = 0

    for batch in test_dataloader:

        for key in batch:
            batch[key] = batch[key].to(device)

        predictions = model(batch)

        predictions = F.softmax(predictions, dim=1)

        predictions = torch.argmax(predictions,dim=1)

        classes = torch.argmax(batch["label"], dim=1)

        correct += torch.sum(predictions == classes, dtype=torch.int)

    model.train()

#training

def train_epoch(model=None,
          train_dataloader=None,
          valid_dataloader=None,
          optimiser = None,
          train_config=None,
          valid_config=None,
          lr_scheduler=None,
          min_valid_loss=100.,
          model_name="model",
          epoch=0,
          logger=None):
    
    """
    Training Function
    """

    assert(train_dataloader is not None)
    
    assert(model is not None)

    #Training Params

    save_model = train_config.get("save_model", True)
    to_model_keys = ["spec", "eeg", "label"]
    valid_step = valid_config.get("valid_step", 1000)
    verbose_step = train_config.get("verbose_step", 10)
    
    
    loss_fn = nn.KLDivLoss(reduction="batchmean")
    
    model.train()
    
    for itr, batch in tqdm(enumerate(train_dataloader)):

#       print(batch["eeg"].shape)

        #Train One Iteration

        #Load data to GPU

        for key in to_model_keys:

            batch[key] = batch[key].to(device)

        predictions = model(batch)
        
        predictions = F.log_softmax(predictions,dim=1)

        loss = loss_fn(predictions,batch["label"])
        
        optimiser.zero_grad()

        loss.backward()

        optimiser.step()
        
        if lr_scheduler is not None:
            lr_scheduler.step()
        
        if itr%verbose_step==0:
            print(f"Training itr {itr}/{len(train_dataloader)}")
            for param_group in optimiser.param_groups:
                lr = param_group['lr']
                break
            print("Training Loss: ", loss, "Learning Rate:", lr)

            logger.log_metrics({"KLDiv loss":loss, "Learning Rate": lr}, itr+epoch*len(train_dataloader))
        
        #Test
        
        if itr%valid_step==0 and itr>0 and valid_dataloader is not None:
            print("Validation")
            valid_loss = validate(model, valid_dataloader)
            
            if valid_loss < min_valid_loss and save_model:
                min_valid_loss = valid_loss
                save_path = os.path.join(DATA_ROOT,"models")
                os.makedirs(save_path, exist_ok=True)
                torch.save(model.state_dict(), os.path.join(save_path, f"{model_name}_best.pt"))
            logger.log_metrics({"Valid loss":valid_loss}, itr+epoch*len(train_dataloader))
    return min_valid_loss

In [11]:
def train(model=None,
          optimiser=None,
          lr_scheduler=None,
          train_config=None,
          valid_config=None,
          model_name="model",
          logger=None):
    
    train_batch_size = train_config.get("batch_size", 32)
    num_epochs = train_config.get("num_epochs", 10)
    valid_folds = train_config.get("valid_folds", 5)
    valid_batch_size = valid_config.get("batch_size", 32)
    train_workers = train_config.get("workers", 1)
    valid_workers = valid_config.get("workers", 1)

    min_valid_loss = 100

    for epoch in range(num_epochs):

        print("Epoch", epoch)

        #KGFolds
        fold = np.random.randint(valid_folds)
        train_df_fold = train_df[train_df["fold"]!=fold]
        valid_df = train_df[train_df["fold"]==fold]
        train_dataset = HMSDataset(train_df_fold, aug=True)
        valid_dataset = HMSDataset(valid_df, aug=False)

        train_dataloader = DataLoader(
            dataset=train_dataset,
            batch_size = train_batch_size,
            shuffle=True,
            num_workers=train_workers,
            collate_fn=train_dataset.collate_fn
        )

        valid_dataloader = DataLoader(
            dataset=valid_dataset,
            batch_size = valid_batch_size,
            shuffle=True,
            num_workers=valid_workers,
            collate_fn=valid_dataset.collate_fn
        )

        min_valid_loss = train_epoch(model=model,
                    train_dataloader=train_dataloader,
                    valid_dataloader=valid_dataloader,
                    optimiser=optimiser,
                    train_config=train_config,
                    valid_config=valid_config,
                    lr_scheduler=lr_scheduler,
                    min_valid_loss=min_valid_loss,
                    model_name=model_name,
                    epoch=epoch,
                    logger=logger)
        print("Min Valid Loss", min_valid_loss)

In [12]:
model_config = {}

model = CNNDetector(model_config).to(device)

train_config = {
    "batch_size": 32,
    "num_epochs": 5,
    "lr": 1e-3,
    "valid_folds":5,
    "save_model": True,
    "workers": 1,
    "verbose_step": 100,
    "weight_decay": 1e-4,
    "valid_folds": 5,
}

valid_config = {
    "batch_size": 16,
    "workers": 1,
    "valid_step": 500
}

lr = train_config.get("lr", 1e-3)
weight_decay = train_config.get("weight_decay", 0.)
model_name = "efficientnetb0"
num_epochs = train_config.get("num_epochs", 1)
folds = train_config.get("valid_folds", 5)
train_batch_size = train_config.get("batch_size", 32)


logger = TensorBoardLogger(f"logs/{model_name}", name=model_name)

optimiser = torch.optim.Adam(
model.parameters(),
lr=lr,
weight_decay=weight_decay
)

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimiser,
    max_lr=lr,
    epochs=num_epochs,
    steps_per_epoch=int((folds-1)/folds*train_df.shape[0]/train_batch_size)+10
)

torch.cuda.empty_cache()

train(model=model,
      train_config=train_config,
      valid_config=valid_config,
      model_name=model_name,
      optimiser=optimiser,
      lr_scheduler=lr_scheduler,
      logger=logger)

    

Epoch 0


0it [00:00, ?it/s]/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1545: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe')
/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in subtract
2it [00:05,  2.47s/it]

Training itr 0/1202
Training Loss:  tensor(1.3886, device='cuda:0', grad_fn=<DivBackward0>) Learning Rate: 4.0000718651559374e-05


12it [00:07,  6.18it/s]/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
102it [00:18,  8.05it/s]

Training itr 100/1202
Training Loss:  tensor(1.0628, device='cuda:0', grad_fn=<DivBackward0>) Learning Rate: 4.731232458808946e-05


...

Training itr 300/1202
Training Loss:  tensor(0.5884, device='cuda:0', grad_fn=<DivBackward0>) Learning Rate: 0.00010365180448417455


402it [00:57,  8.05it/s]

Training itr 400/1202
Training Loss:  tensor(0.5460, device='cuda:0', grad_fn=<DivBackward0>) Learning Rate: 0.00015099686451240733


457it [01:04,  7.10it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:54                                                                                   │
│ in train:55                                                                                      │
│ in train_epoch:101                                                                               │
│                                                                                                  │
│ /home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/torch/nn/modules/module.p │
│ y:1130 in _call_impl                                                                             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│ in forward:31                                                                                    │
│                                                                                                  │
│ /home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/torch/nn/modules/module.p │
│ y:1130 in _call_impl                                                                             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/torchvision/models/effici │
│ entnet.py:355 in forward                                                                         │
│                                                                                                  │
│    352 │   │   return x                                                                          │
│    353 │                                                                                         │
│    354 │   def forward(self, x: Tensor) -> Tensor:                                               │
│ ❱  355 │   │   return self._forward_impl(x)                                                      │
│    356                                                                                           │
│    357                                                                                           │
│    358 def _efficientnet(                                                                        │
│                                                                                                  │
│ /home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/si